<a href="https://colab.research.google.com/github/kimwoonggon/kostat_giro_economics/blob/master/(%EB%B0%B0%ED%8F%AC%EC%9A%A9)%EA%B2%BD%EC%9D%B8%EC%B2%AD_%EA%B1%B4%EC%84%A4_%EB%AF%BC%EA%B0%84%EC%9E%90%EB%A3%8C_%EC%88%98%EC%A7%91%EC%8B%9C%EC%8A%A4%ED%85%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 경인청 건설경기동향조사 민간자료 자동 수집 시스템
* 조사담당자와 검색할 기업체명을 업로드하면 자동으로 자료 수집
* 수집된 기사를 AI를 활용하여 기사 필터링
* 배포의 용이성을 위해 코드로 배포
* Python Jupyter notebook 및 구글 Colaboratory 에서 실행 가능
* AI 필터링 시에는 GPU 사용을 권장

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

# 건설경기동향조사 수집 명부 업로드
* 칼럼이 기업체명, 담당자가 되도록 한다
* 나라통계에서 다운 받은 것을 csv 파일로 저장해야 함

In [ ]:
from google.colab import files
files.upload()

# 자료 수집을 위한 각종 설정

In [ ]:
# 네이버 개발자센터에서 발급받은 API ID, API PASSWORD
API_ID = 'XYtO5E39pr5gM9kmseZg'
API_PASSWORD = 'G7tIG_RE3t'

# 수집 일자
DAYS = 30

# 기초 필터
FILTER1 = ['일본', '중국', '러시아', '대만', '필리핀', '브루나이', '인도네시아', '동티모르', '파푸아뉴기니',
           '싱가포르', '말레이시아', '캄보디아', '베트남', '라오스', '태국', '미얀마', '방글라데시', '부탄', '네팔',
           '스리랑카', '인도', '몰디브', '파키스탄', '카자흐스탄', '키르키스스탄', '우즈베키스탄', '투르크메니스탄',
           '아프가니스탄', '이란', '이라크', '쿠웨이트', '바레인', '아랍에미리트', 'UAE', '시리아', '터키', '레바논',
           '요르단', '이스라엘', '사우디아라비아', '카타르', '오만', '예멘', '이집트', '알제리', '아시안','중동', '동남아', '사우디']
FILTER2 = ['체코', '폴란드', '유럽', '스페인', '이탈리아']
FILTER3 = ['멕시코', '남미', '아프리카', '에티오피아', '나이지리아','파나마', '모잠비크']

FILTERS = FILTER1 + FILTER2 + FILTER3

#수집한 기사를 저장할 파일 이름
SAVED_FILE_NAME = "경제조사과_건설수집자료.xlsx"

# 자료 수집 함수 정의

In [ ]:
## 이 버전은 담당자별로 자료를 수집하여 담당자가 조사하는 건설사 별로 자료를 수집하고 분류함
## 최종 수정 날짜 : '20. 11. 2.
import os
import urllib.request
import sys
import datetime
import time
import json
import re
import collections
import glob

import numpy as np
import pandas as pd



class gunseol_crawl:    
    def __init__(self, file_path, api_id=API_ID, api_password=API_PASSWORD):
        self.total_data = pd.DataFrame(columns=['날짜', '제목', '기사요약', '링크', '담당자', '사업체명'])
        self.file_path = file_path
        self.api_id = api_id
        self.api_password = api_password
        
        self.user_search_lists = self.load_gunseol_list()

        for self.user_search_list in self.user_search_lists:
                
            print(self.user_search_list[0]+'님의 건설경기 자료를 수집하겠습니다\n')

            print(self.user_search_list)
            self.crawl_news_data(self.user_search_list[1:], self.user_search_list[0])
            print('%s님 수고하셨습니다.' % self.user_search_list[0])
        
        self.save_to_excel()


    def load_gunseol_list(self):
        gunseol_list = pd.read_csv(self.file_path, encoding='cp949', engine='python')
        total_gunseol_list = []
        name_list = gunseol_list['담당자'].unique().tolist()
        for name in name_list:
            gun = gunseol_list.loc[gunseol_list['담당자'] == name_list[name_list.index(name)]]['기업체명'].tolist()
            re_gun = []
            for i in gun:
                del_letter = re.compile('\((주)\)')
                i = del_letter.sub("", i)

                re_gun.append(i)
            re_gun.insert(0, name)
            total_gunseol_list.append(re_gun)

        return total_gunseol_list

    def get_naver_connection(self, url):

        req = urllib.request.Request(url)
        req.add_header('X-Naver-Client-Id', self.api_id)
        req.add_header('X-Naver-Client-Secret', self.api_password)

        try:
            response = urllib.request.urlopen(req)

            if response.getcode() == 200:
                return response.read().decode('utf-8')

        except Exception as e:
            return None


    def get_naver_json(self, kind, search_text, page_start, display_num):
        base = 'https://openapi.naver.com/v1/search'
        node = '/%s.json' % kind
        parameters = '?query=%s&start=%s&display=%s' % (urllib.parse.quote(search_text), page_start, display_num)
        url = base + node + parameters


        raw_json = self.get_naver_connection(url)

        if raw_json == None:
            return None
        else:
            return json.loads(raw_json)

    def process_data(self, post, jsonResult):

        pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
        if (datetime.datetime.now() - pDate).total_seconds() > int(86400 * DAYS):
            return None
        else:
            pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')
            title = post['title']

        del_letters1 = re.compile('[<b>|</b>|]')
        del_letters2 = re.compile('[&quot]')

        title = del_letters1.sub("",title)
        title = del_letters2.sub("",title)
        description = post['description']
        description = del_letters1.sub("",description)
        description = del_letters2.sub("",description)

        link = post['originallink']

        description, title = self.filter_article(description=description, title=title)
        if description is not None and title is not None:

            sys.stdout.write(title+"\n")
            jsonResult.append({'제목':title, '기사요약':description, '링크':link, '날짜':pDate})

        return None

    def filter_article(self, description, title):

        for filter in FILTERS:
            if filter in description or filter in title:

                return None, None

        return description, title

    def crawl_news_data(self, user_search_list, user_name):

        display_count = 100
        kind = 'news'

        for search in user_search_list:
            self.jsonResult = []
            pre_search_text = '"수주"'
            real_search_text = search
            search_text = real_search_text + ' ' + pre_search_text
            jsonSearch = self.get_naver_json(kind, search_text, 1, display_count)

            while ((jsonSearch != None) and (jsonSearch['display'] != 0)):

                for post in jsonSearch['items']:
                    self.process_data(post, self.jsonResult)

                nStart = jsonSearch['start'] + jsonSearch['display']
                jsonSearch = self.get_naver_json(kind, search_text, nStart, display_count)

            with open("%s.json" % search, 'w', encoding='utf8') as f:
                result_json = json.dumps(self.jsonResult, indent=4, sort_keys=True, ensure_ascii=False)
                f.write(result_json)

            data = pd.read_json("%s.json" % search, encoding='utf-8')
            data = pd.DataFrame(data, columns = ['날짜', '제목', '기사요약', '링크'])

            data['담당자'] = user_name
            data['사업체명'] = real_search_text

            self.total_data = pd.concat([self.total_data, data], axis=0)
            del self.jsonResult

        jsonFileList = glob.glob('*.json')
        for jsonFile in jsonFileList:
            os.remove(jsonFile)
        print('-' * 50)
        print('수집 완료!')
        print('-' * 50)

    def save_to_excel(self):
        self.total_data.reset_index(drop=True)
        self.total_data = self.total_data[['담당자','사업체명','날짜', '제목', '기사요약', '링크']]
        self.total_data.to_excel(SAVED_FILE_NAME, index=False)

# 자료 수집 시작
* (예시)명부가 저장된 파일인 (202009)건설명부.csv 입력

In [ ]:
if __name__ == '__main__':
    crawl = gunseol_crawl("(202009)건설명부.csv")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
현대건설, 안정적인 외형성장 전망-SK증권
[입찰 리뷰] 계룡, 설계 경제성 확보… 토목 기술형입찰 첫 승
현대건설, 세종 네이버 제2데이터센터 '각 세종' 1단계 공사 수주
2.5兆 규모 '세종 스마트시티' LG CNS-현대차 2파전
대우건설 신반포 소송 패배로 부담 커, 강남 수주전에 영향 줄 가능성
대연8구역 재개발, 시공사 선정 둘러싼 조합 내 갈등 심화...사업 지연 '경고...
흑석11구역 재개발 설명회에 대형건설사 '총집결'
5000가구 대단지 '남산타운', 리모델링 사업 ‘시동’
[MTN현장+]아시아나항공, 대한항공 인수설의 끝은?
중견 건설사, 이젠 대형사 자회사와도 영역다툼
[더벨]신동아건설, GS 출신 개발 전문가 영입···수도권 이남 노린다
현대重, 조선·건설기계 시장 장악 '초읽기'
[건설오늘] 현대건설, 갑천1 트리풀시티 힐스테이트 분양…한양, 의정부 고산...
서울 '준강남' 흑석11구역, 별들의 전쟁 예고
재건축재개발 4분기 막판 수주전…건설업계 '일년 농사' 판가름
[이슈] 가람환경기술(주), 친환경 신기술로 강소기업 우뚝
;시공사 바뀌는 정비사업장 잡아라;
[뉴스돋보기] 실적 개선된 해외건설 시장, 어디서 누가 활약했나
신영증권 ;현대건설, 3분기 해외 실적 둔화 전망…노량진4구역-남북철도 등 수...
호황기에 재건축 손 놓은 삼성물산, 이재용 승계 위해 인력 감축도 불사했나
대우건설 신반포15차 소송 져, 재건축 공사비 증액 갈등의 기준 되나
[단독] 대연8구역 민원처리비 논란 '포스코건설', 국토부발 위법성 지침에도 ...
연간 분양목표 절반 채운 10대 건설사, 4분기 지방 중소도시 분양에 집중하나
반포3주구 조합, 공사비 증액 놓고 시공사와 갈등
'절치부심' 대우건설, 흑석11구역 수주로 반전 노린다
재건축 조합 ;계약 한달도 안돼 공사비 증액 전례없어; VS 시공사 ;조합 요청...
현대중공업 권오갑의 ‘투트랙 생존전략' 성공할까?
10월 1주 주요 제조업

# 자동으로 수집된 파일 다운받기
* 경제조사과_건설수집자료.xlsx 파일을 저장

In [ ]:
from google.colab import files
files.download(SAVED_FILE_NAME)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 수집 자료 AI 필터링
* 조사에 필요한 기사들만 추출하도록 사전 학습한 AI를 활용하여 기사 필터링

# 사전 학습된 AI 모델 활용하기

In [ ]:
# huggingface 모듈 설치
!pip install transformers

* 필요 모듈 로드

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

import logging
import os
import unicodedata
from shutil import copyfile

from transformers import PreTrainedTokenizer
from google.colab import files

logger = logging.getLogger(__name__)

VOCAB_FILES_NAMES = {"vocab_file": "tokenizer_78b3253a26.model",
                     "vocab_txt": "vocab.txt"}

PRETRAINED_VOCAB_FILES_MAP = {
    "vocab_file": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/tokenizer_78b3253a26.model",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/tokenizer_78b3253a26.model",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/tokenizer_78b3253a26.model"
    },
    "vocab_txt": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/vocab.txt",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/vocab.txt",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/vocab.txt"
    }
}

PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {
    "monologg/kobert": 512,
    "monologg/kobert-lm": 512,
    "monologg/distilkobert": 512
}

PRETRAINED_INIT_CONFIGURATION = {
    "monologg/kobert": {"do_lower_case": False},
    "monologg/kobert-lm": {"do_lower_case": False},
    "monologg/distilkobert": {"do_lower_case": False}
}

SPIECE_UNDERLINE = u'▁'


class KoBertTokenizer(PreTrainedTokenizer):
    """
        SentencePiece based tokenizer. Peculiarities:
            - requires `SentencePiece <https://github.com/google/sentencepiece>`_
    """
    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES

    def __init__(
            self,
            vocab_file,
            vocab_txt,
            do_lower_case=False,
            remove_space=True,
            keep_accents=False,
            unk_token="[UNK]",
            sep_token="[SEP]",
            pad_token="[PAD]",
            cls_token="[CLS]",
            mask_token="[MASK]",
            **kwargs):
        super().__init__(
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            **kwargs
        )

        # Build vocab
        self.token2idx = dict()
        self.idx2token = []
        with open(vocab_txt, 'r', encoding='utf-8') as f:
            for idx, token in enumerate(f):
                token = token.strip()
                self.token2idx[token] = idx
                self.idx2token.append(token)

        self.max_len_single_sentence = self.max_len - 2  # take into account special tokens
        self.max_len_sentences_pair = self.max_len - 3  # take into account special tokens

        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")

        self.do_lower_case = do_lower_case
        self.remove_space = remove_space
        self.keep_accents = keep_accents
        self.vocab_file = vocab_file
        self.vocab_txt = vocab_txt

        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(vocab_file)

    @property
    def vocab_size(self):
        return len(self.idx2token)

    def __getstate__(self):
        state = self.__dict__.copy()
        state["sp_model"] = None
        return state

    def __setstate__(self, d):
        self.__dict__ = d
        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(self.vocab_file)

    def preprocess_text(self, inputs):
        if self.remove_space:
            outputs = " ".join(inputs.strip().split())
        else:
            outputs = inputs
        outputs = outputs.replace("``", '"').replace("''", '"')

        if not self.keep_accents:
            outputs = unicodedata.normalize('NFKD', outputs)
            outputs = "".join([c for c in outputs if not unicodedata.combining(c)])
        if self.do_lower_case:
            outputs = outputs.lower()

        return outputs

    def _tokenize(self, text, return_unicode=True, sample=False):
        """ Tokenize a string. """
        text = self.preprocess_text(text)

        if not sample:
            pieces = self.sp_model.EncodeAsPieces(text)
        else:
            pieces = self.sp_model.SampleEncodeAsPieces(text, 64, 0.1)
        new_pieces = []
        for piece in pieces:
            if len(piece) > 1 and piece[-1] == str(",") and piece[-2].isdigit():
                cur_pieces = self.sp_model.EncodeAsPieces(piece[:-1].replace(SPIECE_UNDERLINE, ""))
                if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
                    if len(cur_pieces[0]) == 1:
                        cur_pieces = cur_pieces[1:]
                    else:
                        cur_pieces[0] = cur_pieces[0][1:]
                cur_pieces.append(piece[-1])
                new_pieces.extend(cur_pieces)
            else:
                new_pieces.append(piece)

        return new_pieces

    def _convert_token_to_id(self, token):
        """ Converts a token (str/unicode) in an id using the vocab. """
        return self.token2idx.get(token, self.token2idx[self.unk_token])

    def _convert_id_to_token(self, index, return_unicode=True):
        """Converts an index (integer) in a token (string/unicode) using the vocab."""
        return self.idx2token[index]

    def convert_tokens_to_string(self, tokens):
        """Converts a sequence of tokens (strings for sub-words) in a single string."""
        out_string = "".join(tokens).replace(SPIECE_UNDERLINE, " ").strip()
        return out_string

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        """
        Build model inputs from a sequence or a pair of sequence for sequence classification tasks
        by concatenating and adding special tokens.
        A RoBERTa sequence has the following format:
            single sequence: [CLS] X [SEP]
            pair of sequences: [CLS] A [SEP] B [SEP]
        """
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        return cls + token_ids_0 + sep + token_ids_1 + sep

    def get_special_tokens_mask(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
        """
        Retrieves sequence ids from a token list that has no special tokens added. This method is called when adding
        special tokens using the tokenizer ``prepare_for_model`` or ``encode_plus`` methods.
        Args:
            token_ids_0: list of ids (must not contain special tokens)
            token_ids_1: Optional list of ids (must not contain special tokens), necessary when fetching sequence ids
                for sequence pairs
            already_has_special_tokens: (default False) Set to True if the token list is already formated with
                special tokens for the model
        Returns:
            A list of integers in the range [0, 1]: 0 for a special token, 1 for a sequence token.
        """

        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError(
                    "You should not supply a second sequence if the provided sequence of "
                    "ids is already formated with special tokens for the model."
                )
            return list(map(lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0, token_ids_0))

        if token_ids_1 is not None:
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1]

    def create_token_type_ids_from_sequences(self, token_ids_0, token_ids_1=None):
        """
        Creates a mask from the two sequences passed to be used in a sequence-pair classification task.
        A BERT sequence pair mask has the following format:
        0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
        | first sequence    | second sequence
        if token_ids_1 is None, only returns the first portion of the mask (0's).
        """
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            return len(cls + token_ids_0 + sep) * [0]
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]

    def save_vocabulary(self, save_directory):
        """ Save the sentencepiece vocabulary (copy original file) and special tokens file
            to a directory.
        """
        if not os.path.isdir(save_directory):
            logger.error("Vocabulary path ({}) should be a directory".format(save_directory))
            return

        # 1. Save sentencepiece model
        out_vocab_model = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_file"])

        if os.path.abspath(self.vocab_file) != os.path.abspath(out_vocab_model):
            copyfile(self.vocab_file, out_vocab_model)

        # 2. Save vocab.txt
        index = 0
        out_vocab_txt = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_txt"])
        with open(out_vocab_txt, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(self.token2idx.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        "Saving vocabulary to {}: vocabulary indices are not consecutive."
                        " Please check that the vocabulary is not corrupted!".format(out_vocab_txt)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1

        return out_vocab_model, out_vocab_txt

In [ ]:
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

# AI 필터링 모델 정의 및 불러오기

In [ ]:
import tensorflow_addons as tfa
SEQ_LEN = 128

def get_gisa_model():
    model = TFBertModel.from_pretrained("monologg/kobert", from_pt=True)
    token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids')
    mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks')
    segment_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_segment')
    bert_outputs = model([token_inputs, mask_inputs, segment_inputs])

    opt = tfa.optimizers.RectifiedAdam(lr=5.0e-5, total_steps = 3907, warmup_proportion=0.1, min_lr=1e-5, epsilon=1e-08, clipnorm=1.0)
    gisa_drop = tf.keras.layers.Dropout(0.5)(bert_outputs[1])
    gisa_first = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(gisa_drop)
    gisa_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], gisa_first)
    gisa_model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])

    return gisa_model

In [ ]:
path = "/content/gdrive/My Drive/Colab Notebooks/건설경기동향조사"

gisa_model = get_gisa_model()
# 사전 학습한 AI 필터링 모델 불러오기 (개인 드라이브에 별도 저장되어 있음)
gisa_model.load_weights(path+"/gunseol.h5")

# 수집 데이터 필터링
* 방금 전 수집했던 "경제조사과_건설수집자료" 업로드

In [ ]:
files.upload()

In [ ]:
gisa = pd.read_excel("경제조사과_건설수집자료.xlsx")
gisa['제목_기사요약'] = gisa['제목'].astype(str) + " " + gisa['기사요약'].astype(str)
gisa.head(5)

,담당자,사업체명,날짜,제목,기사요약,링크,제목_기사요약
0,최병렬,삼환기업,2020-10-29 15:15:00,"삼부토건·코디엠·휴림로봇·우진, 이낙연 대선 테마로 '반짝'?",전 삼환기업 대표를 회사의 사내이사로 선임한다고 밝혔다. 이에 대선 이후 수혜 기대...,http://www.thevaluenews.co.kr/news/view.php?id...,"삼부토건·코디엠·휴림로봇·우진, 이낙연 대선 테마로 '반짝'? 전 삼환기업 대표를 ..."
1,최병렬,삼환기업,2020-10-29 08:16:00,"삼부토건, 거제에 전통한옥호텔 짓는다","7000억원, 수주잔고는 약 1조 1000억원을 예상하고 있다. 특히 지난 22일에...",http://www.edaily.co.kr/news/newspath.asp?news...,"삼부토건, 거제에 전통한옥호텔 짓는다 7000억원, 수주잔고는 약 1조 1000억원..."
2,최병렬,삼환기업,2020-10-26 08:52:00,"이낙연 동생 이계연, 삼부토건 사장 선임에 주가 관심받나",지난해 11월 SM그룹 우오현 회장이 30사단 명예사단장 자격으로 장병들을 사열한 ...,http://moneys.mt.co.kr/news/mwView.php?no=2020...,"이낙연 동생 이계연, 삼부토건 사장 선임에 주가 관심받나 지난해 11월 SM그룹 우..."
3,최병렬,삼환기업,2020-10-25 12:58:00,"이낙연 대표 동생 이계연 씨, 삼부토건 사장 선임",삼부토건 CI 삼부토건은 지난 22일 공시를 통해 이계연 전 삼환기업 대표를 사내이...,http://www.segyebiz.com/newsView/2020102550993...,"이낙연 대표 동생 이계연 씨, 삼부토건 사장 선임 삼부토건 CI 삼부토건은 지난 2..."
4,최병렬,삼환기업,2020-10-24 10:38:00,"삼부토건, '이낙연 테마주' 남선알미늄과 희비 엇갈려",지난 2018년 남선알미늄은 삼환기업의 이계연 사장이 이 대표의 동생이라는 이유로 ...,http://www.fnnews.com/news/202010241036591190,"삼부토건, '이낙연 테마주' 남선알미늄과 희비 엇갈려 지난 2018년 남선알미늄은 ..."


* 텍스트가 인공지능이 인식할 수 있도록 숫자로 변환
* GPU 활용을 권장함

In [ ]:
def predict_convert_data(data_df):
    global tokenizer
    tokens, masks, segments = [], [], []
    
    for i in tqdm(range(len(data_df))):
        token = tokenizer.encode(gisa[DATA_COLUMN].iloc[i].values[0], max_length=SEQ_LEN, truncation=True, padding='max_length')
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        segment = [0]*SEQ_LEN

        tokens.append(token)
        segments.append(segment)
        masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def predict_load_data(pandas_dataframe):
    data_df = pandas_dataframe
    #data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_x = predict_convert_data(data_df)
    return data_x

# 기사를 숫자로 변환
DATA_COLUMN = ['제목_기사요약']
SEQ_LEN = 128

# 시간 20분 가량 소요됨(GPU 사용 시 3분 내로 가능)
gisa_num_inputs = predict_load_data(gisa)
gisa_predict = gisa_model.predict(gisa_num_inputs, batch_size=16)

100%|██████████| 5973/5973 [00:07<00:00, 752.43it/s]


* 확률이 0.5 이상이면 기사여부 = 1, else 0 으로 필터링

In [ ]:
gisa['기사여부'] = np.round(np.squeeze(gisa_predict),0)
gisa['확률'] = np.squeeze(gisa_predict)

#필터링된 파일 저장
del gisa['제목_기사요약']
gisa.to_excel("기사수집_필터링완료.xlsx", index=False)

* 필터링이 잘 되었는지 기사 확인
* 기사여부 1 => 필요 기사

In [ ]:
gisa[gisa['기사여부']==1].head(5)

,담당자,사업체명,날짜,제목,기사요약,링크,기사여부,확률
34,최병렬,한양,2020-10-27 03:02:00,"[단신]KB오토텍, 벤츠 전기차에 공기정화장치 공급 外",‘이오나이저’를 수주해 공급한다고 26일 밝혔다. 이오나이저는 음이온을 발생시켜 공...,https://www.donga.com/news/article/all/2020102...,1.0,0.864319
43,최병렬,한양,2020-10-26 14:47:00,"한양, 여수 LNG 허브 터미널 기초 건설공사 착공",한편 올해 한양은 국내 최대 솔라시도 태양광(98MW 태양광+306MWh ESS)을...,http://www.e2news.com/news/articleView.html?id...,1.0,0.745102
94,최병렬,한양,2020-10-12 13:33:00,"LG CNS 컨소시엄, 2조5000억원 규모 '세종 스마트시티' 사업 수주",사업을 수주했다. LG CNS가 이끄는 컨소시엄이 첨단 IT 기술을 보유한 대·중·...,http://www.koreastocknews.com/news/articleView...,1.0,0.997256
104,최병렬,한양,2020-10-09 09:00:00,"LG CNS, 2.5조 규모 세종시 스마트시티 구축사업 수주",끝에 수주했다. 9일 관련 업계 등에 따르면 전날 LG CNS 컨소시엄은 한국토지주...,http://www.newspim.com/news/view/20201009000037,1.0,0.998840
134,최병렬,대림산업,2020-11-02 14:45:00,"대림건설, 2700억 대전 옥계2구역 수주... 창사 첫 정비사업 '1조 클럽' 가입",사진=대림산업 대림건설이 창사 후 처음으로 도시정비사업 수주 1조원을 돌파했다고 2...,http://www.meconomynews.com/news/articleView.h...,1.0,0.982119


* 최종 필터링 된 파일 다운로드

In [ ]:
from google.colab import files
files.download("기사수집_필터링완료.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>